In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu
import numpy as np

# Load the two datasets (MCI and NC) from CSV files
file_mci = "/content/drive/MyDrive/radiomic_features_bratshgg_dec22.csv"  # Replace with your actual file path
file_nc = "/content/drive/MyDrive/radiomic_features_bratslgg_dec22.csv"    # Replace with your actual file path

data_mci = pd.read_csv(file_mci)
data_nc = pd.read_csv(file_nc)

# Ensure that the data contains only numerical features
# If the first column is an identifier, drop it
if data_mci.columns[0].lower() in ["id", "subject_id"]:
    data_mci = data_mci.drop(columns=data_mci.columns[0])
    data_nc = data_nc.drop(columns=data_nc.columns[0])

def extract_number(array_string):
    if isinstance(array_string, str) and 'array' in array_string:
        return float(array_string.split('(')[1].split(')')[0])
    else:
        return array_string

# Apply the function to each column
for column in data_mci .columns:
    data_mci [column] = data_mci [column].apply(extract_number)

for column in data_nc .columns:
    data_nc [column] = data_nc [column].apply(extract_number)
# Perform Mann-Whitney U test for each feature
results = []
for feature in data_mci.columns:
    # Extract feature values for MCI and NC groups
    mci_values = data_mci[feature].values
    nc_values = data_nc[feature].values

    # Perform the Mann-Whitney U test
    stat, p_value = mannwhitneyu(mci_values, nc_values, alternative='two-sided')

    # Append results
    results.append((feature, stat, p_value))

# Convert results to a DataFrame for easy analysis
results_df = pd.DataFrame(results, columns=["Feature", "U-Statistic", "P-Value"])

# Apply a significance threshold (e.g., 0.05) and adjust for multiple comparisons if needed
significance_threshold = 0.05
results_df["Significant"] = results_df["P-Value"] < significance_threshold

# Save results to a CSV file
results_df.to_csv("mann_whitney_results.csv", index=False)

# Display significant features
significant_features = results_df[results_df["Significant"]]
print(f"Significant Features:\n{significant_features}")


Significant Features:
                                 Feature  U-Statistic       P-Value  \
0                           10Percentile       7650.0  3.368980e-32   
1                           90Percentile       7648.0  9.279862e-32   
2                                 Energy       5670.0  4.361295e-08   
3                                Entropy       7650.0  7.136304e-30   
4                     InterquartileRange       7650.0  3.303890e-32   
..                                   ...          ...           ...   
81  SmallDependenceHighGrayLevelEmphasis       7600.0  3.856376e-29   
82   SmallDependenceLowGrayLevelEmphasis       7560.0  1.463789e-28   
83                              Busyness       1492.0  4.383381e-12   
85                            Complexity       7645.0  8.455530e-30   
86                              Strength       2512.0  9.768401e-05   

    Significant  
0          True  
1          True  
2          True  
3          True  
4          True  
..          ...  

In [ ]:
# Type I Error: Simulate the null hypothesis
def calculate_type_I_error(data, num_simulations=425, significance_threshold=0.05):
    all_features = pd.concat([data_mci, data_nc])
    type_I_count = 0

    for _ in range(num_simulations):
        # Randomly split data into two groups
        group1 = all_features.sample(frac=0.5, random_state=_)
        group2 = all_features.drop(group1.index)

        # Perform Mann-Whitney U test for each feature
        for feature in group1.columns:
            stat, p_value = mannwhitneyu(group1[feature], group2[feature], alternative='two-sided')
            if p_value < significance_threshold:
                type_I_count += 1

    type_I_error_rate = type_I_count / (num_simulations * len(data_mci.columns))
    return type_I_error_rate


In [ ]:
# Type II Error: Simulate the alternative hypothesis
def calculate_type_II_error(data_mci, data_nc, num_simulations=1000, significance_threshold=0.05, effect_size=0.5):
    type_II_count = 0

    for _ in range(num_simulations):
        # Introduce an artificial shift (effect size) to simulate the alternative hypothesis
        shifted_mci = data_mci + np.random.normal(effect_size, 0.1, size=data_mci.shape)
        shifted_nc = data_nc.copy()

        # Perform Mann-Whitney U test for each feature
        for feature in data_mci.columns:
            stat, p_value = mannwhitneyu(shifted_mci[feature], shifted_nc[feature], alternative='two-sided')
            if p_value >= significance_threshold:
                type_II_count += 1
                type_II_error_rate = type_II_count / (num_simulations * len(data_mci.columns))
    return type_II_error_rate

In [ ]:
# Calculate errors
type_I_error = calculate_type_I_error(data_mci)
type_II_error = calculate_type_II_error(data_mci, data_nc)

print(f"Type I Error Rate: {type_I_error}")
print(f"Type II Error Rate: {type_II_error}")
print(f"Power of the Test: {1 - type_II_error}")

Type I Error Rate: 0.026450304259634887
Type II Error Rate: 0.011494252873563218
Power of the Test: 0.9885057471264368
